In [1]:
from pyspark.sql import SparkSession
from delta import *

builder = (
    SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/nil/anaconda3/envs/spark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nil/.ivy2/cache
The jars for the packages stored in: /home/nil/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4cb5ceda-146d-46e4-acdd-4681507952b3;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 269ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

24/02/16 14:15:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


-  Create Delta table

In [2]:
spark.sql(
    "CREATE TABLE employee_table(emp_id INT,emp_Name STRING, gender STRING, salary INT,dept STRING) USING DELTA"
)

24/02/16 14:18:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[]

In [3]:
spark.sql("show tables").show(truncate=False)

+---------+--------------+-----------+
|namespace|tableName     |isTemporary|
+---------+--------------+-----------+
|default  |employee_table|false      |
+---------+--------------+-----------+



- create table if not exists

In [4]:
spark.sql(
    "CREATE TABLE IF NOT EXISTS employee_table(emp_id INT,emp_Name STRING, gender STRING, salary INT,dept STRING) USING DELTA"
)

DataFrame[]

In [5]:
spark.sql("show tables").show(truncate=False)

+---------+--------------+-----------+
|namespace|tableName     |isTemporary|
+---------+--------------+-----------+
|default  |employee_table|false      |
+---------+--------------+-----------+



- 2nd way to create table

In [6]:
DeltaTable.createIfNotExists(spark).tableName("employee_table").addColumn(
    "emp_id", "INT"
).addColumn("emp_name", "STRING").addColumn("gender", "STRING").addColumn(
    "salary", "INT"
).addColumn(
    "Dept", "STRING"
).property(
    "description", "table created for demo purpose"
).location(
    "/mnt/sda3/iNeuron/Data Engineering/pySpark/Delta_Lake/delta_lake_01"
).execute()

In [7]:
spark.sql("show tables").show(truncate=False)

+---------+--------------+-----------+
|namespace|tableName     |isTemporary|
+---------+--------------+-----------+
|default  |employee_table|false      |
+---------+--------------+-----------+



In [9]:
DeltaTable.createIfNotExists(spark).tableName("employee_table_01").addColumn(
    "emp_id", "INT"
).addColumn("emp_name", "STRING").addColumn("gender", "STRING").addColumn(
    "salary", "INT"
).addColumn(
    "Dept", "STRING"
).property(
    "description", "table created for demo purpose"
).execute()

In [10]:
spark.sql("show tables").show(truncate=False)

+---------+-----------------+-----------+
|namespace|tableName        |isTemporary|
+---------+-----------------+-----------+
|default  |employee_table   |false      |
|default  |employee_table_01|false      |
+---------+-----------------+-----------+



In [12]:
# Describe the table details
spark.sql("DESCRIBE DETAIL employee_table_01").show(truncate=False)

+------+------------------------------------+---------------------------------------+-----------+-----------------------------------------------------------------------------------+-----------------------+-----------------------+----------------+--------+-----------+-----------------------------------------------+----------------+----------------+------------------------+
|format|id                                  |name                                   |description|location                                                                           |createdAt              |lastModified           |partitionColumns|numFiles|sizeInBytes|properties                                     |minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+---------------------------------------+-----------+-----------------------------------------------------------------------------------+-----------------------+-----------------------+----------------+----

In [13]:
# Describe the table details
spark.sql("DESCRIBE DETAIL employee_table").show(truncate=False)

+------+------------------------------------+------------------------------------+-----------+--------------------------------------------------------------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name                                |description|location                                                                        |createdAt              |lastModified           |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+------------------------------------+-----------+--------------------------------------------------------------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|del

- 3rd: Using Dataframe

In [15]:
employee_data = [
    (100, "Stephen", "M", 2000, "IT"),
    (200, "Philipp", "M", 8000, "HR"),
    (300, "Laura", "F", 6000, "SALES"),
]
schema = ["emp_id", "emp_name", "gender", "salary", "dept"]

df = spark.createDataFrame(data=employee_data, schema=schema)
df.show()

+------+--------+------+------+-----+
|emp_id|emp_name|gender|salary| dept|
+------+--------+------+------+-----+
|   100| Stephen|     M|  2000|   IT|
|   200| Philipp|     M|  8000|   HR|
|   300|   Laura|     F|  6000|SALES|
+------+--------+------+------+-----+



In [16]:
# create table in the metastore using dataframe's schema and write data to it
df.write.format("delta").saveAsTable("employee_demo_01")

In [17]:
spark.sql("SELECT * FROM employee_demo_01").show(truncate=False)

+------+--------+------+------+-----+
|emp_id|emp_name|gender|salary|dept |
+------+--------+------+------+-----+
|300   |Laura   |F     |6000  |SALES|
|200   |Philipp |M     |8000  |HR   |
|100   |Stephen |M     |2000  |IT   |
+------+--------+------+------+-----+

